In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
pip install pyspark

using pyspark FIFFA 2023.csv dataset.

In [12]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('fifadata').getOrCreate()

In [27]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [13]:
type(spark)

pyspark.sql.session.SparkSession

In [36]:
df = spark.read.csv("/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/LabTest/FIFA2022 Final.csv", header=True, inferSchema=False)
df.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|
|     Uruguay|    Portugal|   11|              3|        40|   403|           73|   16|           2|        0|      

In [26]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Opponent: string (nullable = true)
 |-- Shots: string (nullable = true)
 |-- Shots on Target: string (nullable = true)
 |-- Possession: string (nullable = true)
 |-- Passes: string (nullable = true)
 |-- Pass accuracy: string (nullable = true)
 |-- Fouls: string (nullable = true)
 |-- Yellow Cards: string (nullable = true)
 |-- Red Cards: string (nullable = true)
 |-- OffSides: string (nullable = true)
 |-- Corners: string (nullable = true)
 |-- Goals For: string (nullable = true)
 |-- Goals against: string (nullable = true)
 |-- Result: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Extra Time: string (nullable = true)
 |-- For Goal Extra: string (nullable = true)
 |-- Against Goal Extra: string (nullable = true)
 |-- Tie braker: string (nullable = true)



No Of Matches

In [128]:
df.count()

126

show all teams

In [115]:
df.select("Team").distinct().count()

32

In [119]:
df.select("Team").distinct().show(32)

+------------+
|        Team|
+------------+
|     Senegal|
|     Germany|
|      France|
|     Cannada|
|       Wales|
|   Argentina|
|     Belgium|
|       Qatar|
|       Ghana|
|     Croatia|
|    Camaroon|
|       Spain|
|     Denmark|
|        Iran|
|     Morocco|
|         USA|
| South Korea|
|     Uruguay|
|      Mexico|
|     Tunisia|
|Saudi Arabia|
| Switzerland|
|      Brazil|
|      Ecador|
|       Japan|
|     England|
|      Poland|
|    Portugal|
|   Australia|
|  Costa Rica|
|      Serbia|
| Netherlands|
+------------+



Show what are the Game Stages

In [111]:
df.select("Stage").distinct().show(100)

+-----+
|Stage|
+-----+
|  R16|
|    F|
|   QF|
|Group|
|   SF|
|Third|
+-----+



Q1) Add column 'Group' and populate the values based on countries

In [97]:
df1 = df.withColumn('Group',
    when(col('Team').isin('Qatar', 'Ecador', 'Senegal', 'Netherlands'), 'A')
    .when(col('Team').isin('England', 'Iran', 'USA', 'Wales'), 'B')
    .when(col('Team').isin('Argentina', 'Saudi Arabia', 'Mexico', 'Poland'), 'C')
    .when(col('Team').isin('France', 'Australia', 'Denmark', 'Tunisia'), 'D')
    .when(col('Team').isin('Spain', 'Costa Rica', 'Germany', 'Japan'), 'E')
    .when(col('Team').isin('Belgium', 'Cannada', 'Morocco', 'Croatia'), 'F')
    .when(col('Team').isin('Brazil', 'Serbia', 'Switzerland', 'Camaroon'), 'G')
    .when(col('Team').isin('Portugal', 'Ghana', 'Uruguay', 'South Korea'), 'H')
    .otherwise('Unknown')
)

df1.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-----+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|Group|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-----+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|    H|
|     Uruguay|    Portugal|   11|              3|        40|   403|           73|   16|     

 Q2) Add column 'Total Goals' representing total goals scored in the match

In [99]:
df2 = df1.withColumn("Total Goals", df1["Goals For"] + df1["Goals Against"])
df2.show()

+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-----+-----------+
|        Team|    Opponent|Shots|Shots on Target|Possession|Passes|Pass accuracy|Fouls|Yellow Cards|Red Cards|OffSides|Corners|Goals For|Goals against|Result|Stage|Extra Time|For Goal Extra|Against Goal Extra|Tie braker|Group|Total Goals|
+------------+------------+-----+---------------+----------+------+-------------+-----+------------+---------+--------+-------+---------+-------------+------+-----+----------+--------------+------------------+----------+-----+-----------+
|    Portugal|     Uruguay|   15|              3|        60|   603|           83|   10|           3|        0|       3|      6|        2|            0|   Won|Group|        NO|            NA|                NA|        NA|    H|        2.0|
|     Uruguay|    Portugal|   11|           

Q3) Find out total goals scored in every group.

In [123]:
df3 = df2.groupBy("Group").agg(sum("Total Goals").alias("Total Goals")).orderBy("Group", asecnding = True)
df3.show()

+-----+-----------+
|Group|Total Goals|
+-----+-----------+
|    A|       41.0|
|    B|       42.0|
|    C|       42.0|
|    D|       36.0|
|    E|       46.0|
|    F|       22.0|
|    G|       56.0|
|    H|       48.0|
+-----+-----------+



Q4) Which country committed more fouls during group stage matches?  

In [121]:
df4 = df2.filter(col('Stage') == 'Group').groupBy("Team").agg(sum("Fouls").alias("Total Fouls")).orderBy(desc("Total Fouls"))
df4.show(40)

+------------+-----------+
|        Team|Total Fouls|
+------------+-----------+
|Saudi Arabia|       56.0|
|      Serbia|       56.0|
|      Mexico|       51.0|
|      Ecador|       50.0|
| Netherlands|       47.0|
|       Ghana|       46.0|
|       Japan|       45.0|
| Switzerland|       43.0|
|    Camaroon|       40.0|
|         USA|       40.0|
|     Tunisia|       40.0|
|      Poland|       39.0|
|        Iran|       38.0|
|   Australia|       37.0|
|     Senegal|       36.0|
|       Wales|       35.0|
|     Uruguay|       34.0|
|    Portugal|       34.0|
|   Argentina|       33.0|
| South Korea|       32.0|
|       Qatar|       31.0|
|      Brazil|       31.0|
|     Morocco|       30.0|
|     Cannada|       28.0|
|     Denmark|       28.0|
|       Spain|       27.0|
|     England|       27.0|
|     Germany|       26.0|
|  Costa Rica|       23.0|
|     Belgium|       20.0|
|     Croatia|       18.0|
|      France|       15.0|
+------------+-----------+



Q5) Which team got more Yellow cards in a match? Who was the opponent?

In [131]:
df5 = df2.groupBy("Team", "Opponent").agg(sum("Yellow Cards").alias("Total Yellow Cards")).orderBy(desc("Total Yellow Cards"))
df5.show(126)

+------------+------------+------------------+
|        Team|    Opponent|Total Yellow Cards|
+------------+------------+------------------+
|   Argentina| Netherlands|               8.0|
|      Serbia| Switzerland|               7.0|
|Saudi Arabia|   Argentina|               6.0|
|Saudi Arabia|      Mexico|               6.0|
| Netherlands|   Argentina|               6.0|
|       Qatar|      Ecador|               5.0|
|   Argentina|      France|               5.0|
|     Uruguay|       Ghana|               5.0|
|       Ghana|    Portugal|               4.0|
|      Mexico|   Argentina|               4.0|
|     Cannada|     Morocco|               4.0|
| Switzerland|      Serbia|               4.0|
|      Serbia|    Camaroon|               4.0|
|    Camaroon|      Serbia|               4.0|
|         USA|       Wales|               4.0|
|    Camaroon|      Brazil|               3.0|
|      France|     England|               3.0|
|     Senegal|       Qatar|               3.0|
|      Poland

In [132]:
df5.count()

122

Q6) Which team/s scored more goals in group stages?

In [112]:
df6 = df2.filter(col('Stage') == 'Group').groupBy("Team").agg(sum("Goals For").alias("Total Goals")).orderBy(desc("Total Goals"))
df6.show(40)

+------------+-----------+
|        Team|Total Goals|
+------------+-----------+
|       Spain|        9.0|
|     England|        9.0|
|      Serbia|        8.0|
|    Camaroon|        7.0|
|     Germany|        6.0|
|      France|        6.0|
|    Portugal|        6.0|
|     Senegal|        5.0|
|   Argentina|        5.0|
|       Ghana|        5.0|
| Netherlands|        5.0|
|        Iran|        4.0|
| South Korea|        4.0|
| Switzerland|        4.0|
|      Ecador|        4.0|
|       Japan|        4.0|
|Saudi Arabia|        3.0|
|      Brazil|        3.0|
|  Costa Rica|        3.0|
|     Morocco|        2.0|
|     Uruguay|        2.0|
|      Mexico|        2.0|
|      Poland|        2.0|
|   Australia|        2.0|
|     Cannada|        1.0|
|       Wales|        1.0|
|     Belgium|        1.0|
|       Qatar|        1.0|
|     Denmark|        1.0|
|         USA|        1.0|
|     Tunisia|        1.0|
|     Croatia|        0.0|
+------------+-----------+

